# Modul 08 Large Language Models und Retrieval Augmented Generation

In diesem Notebook lernst du zuerste unterschiedliche Anwendungsfälle von LLMs kennen. Anschließend wirst für ein vorgegebenes Szenario selbst einen Wartungsassistenten implementieren. 

## Was ist der Anwendungsfall

<div class="alert alert-block alert-info">
<b>Bevor es mit der Umsetztung losgeht machst du dir immer Gedanken darüber welches <i>Problem </i> du lösen möchtest. </b> Aus deinem Problem ergibt sich nämlich welche Technik die richtige ist, um dieses zu lösen. Denn: Es gibt viele Arten von KI-Algorithmen und noch mehr Arten diese anzuwenden. Nicht jeder Algorithmus bzw. jede Technik passt zu jedem Problem! </div>

Damit die Anwendung von LLMs Sinn macht, muss das Problem zunächst mit **Text** zu tun haben. Doch dann stellen sich weitere Fragen. Denn es gibt viele Arten von LLMs die auch unterschiedlich und eingesetzt werden. Einige davon haben wir auf den folgenden Karteikarten beschrieben.


<div class="alert alert-block alert-Info">
<b> QUIZ: </b> Führe die nächste Codezelle aus. Dann werden dir digitale Karteikarten angezeigt. Durch anklicken der Karte dreht sich sich auf die andere Seite. Mit einem Klick auf das Feld <i>Next</i> (unten rechts) kannst du zur nächsten Karte wechseln.

In [13]:
#!pip install jupyterquiz
from jupytercards import display_flashcards
from helpers.fc_helpers import load_flashcards

fc = load_flashcards('fc')
display_flashcards(fc)

<IPython.core.display.Javascript object>

Du weißt bereits, dass Neuronale Netze generell **sehr lang** zum Training brauchen und, dass LLMs auch meistens nicht 'online' lernen. Sie werden in einer gewissen Phase trainiert und machen dann nur noch Vorhersagen. 

D.h. ein LLM mit Chatbot fine-tuning beantwortet sämtliche Anfragen aus seinen eigenen 'Gewichten' heraus. Diese sind durch das Training eingestellt, spiegeln also in gewisser Weise statistische Regelhaftigkeiten in den Trainingsdaten wieder. 

<img src="pictures/LLM_diagramm.png" alt="Platzhalter, Bild kann nicht angezeigt werden.">

Für viele Anwendungen ist das kein Problem. Obwohl sich Sprache ständig verändert, geschieht das nicht so schnell, dass die Modelle nach wenigen Monaten unbrauchbar werden. Für andere Anwendungen ist es allerdings ungünstig:




**Nehmen wir folgendes Beispiel:**

Du willst wissen wer der Premierminister von Armenien ist & dein Onkel hat zu Armenien mal einen Kurs besucht. Also stellst du ihm die Frage: *"Wer ist Premierminister von Armenien?"* Er antwortet: *"Andranik Markarjan von der republikanischen Partei ist der Premier von Armenien."* Die Antwort klingt überzeugend und plausibel, vor allem wenn man selbst keine Ahnung von diesem Land hat. 

<div class="alert alert-block alert-success">
<b>Frage:</b> Woher weiß ich wie verlässlich die Antwort ist? Wie lässt sich diese Information überprüfen? Überlegt gemeinsam und macht Notizen! </div>


Eine einfache online Suche zeigt, wo der Fehler liegt. Die Schulzeit des Onkels ist schon 20 Jahr her, zu dieser Zeit hat er zuletzt etwas über armenische Politiker gelernt. Damals war Andranik Markarjan wirklich Premierminister. Aber inzwischen stimmt das nicht mehr. Denn, wer Premierminister oder Kanzlerin eines Landes ist, ändert sich alle paar Jahre. Aus den Namen der früheren Präsidenten lässt sich nicht auf verlässlich auf den aktuellen Namen schließen.

<div class="alert alert-block alert-danger">
<b>Achtung:</b> Verwechslungsgefahr! Weil LLMs meistens schön formulierte und präzise klingende Antworten auf unsere Fragen erstellen, verwenden viele Nutzende sie als Suchmaschinen. </div>




<div id="Frage1" class="Frage1" style="display:none">

'{"question": "Was k\\u00f6nnten wir also tun, damit der Onkel eine gute Antwort gibt? Nicht nur zu diesen, sondern auch zu weiteren Themen.", "type": "multiple_choice", "answers": [{"answer": "Wir k\\u00f6nnten den Onkel dauerhaft in die Schule schicken, sodass er immer mit dem neuesten Wissen lernt.", "correct": false, "feedback": "Das w\\u00e4re sehr aufwendig und ineffektiv."}, {"answer": "Wir k\\u00f6nnten dem Onkel beibringen die Bibliothek zu benutzen und immer nur zu Antworten, wenn er dort Information zu unserer Frage findet.", "correct": true}]}'

</div>

<div id="Frage2" class="Frage2" style="display:none">
'eyJxdWVzdGlvbiI6ICJXZWxjaGVzIFN0YXRlbWVudCBpc3QgcmljaHRpZz8iLCAidHlwZSI6ICJtdWx0aXBsZV9jaG9pY2UiLCAiYW5zd2VycyI6IFt7ImFuc3dlciI6ICJEYXMgTExNIHNvbGx0ZSBhdWYgYWx0ZW4gV2FydHVuZ3NkYXRlbiB1bmQgSGFuZGJcdTAwZmNjaGVybiBnZWZpbmV0dW5lZCB3ZXJkZW4iLCAiY29ycmVjdCI6IGZhbHNlLCAiZmVlZGJhY2siOiAiRGFzIHdcdTAwZmNyZGUgbnVyIGZcdTAwZmNyIGVpbmVuIEt1cnplbiBaZWl0cmF1bSBmdW5rdGlvbmllcmVuLiBTb2JhbGQgc2ljaCBldHdhcyBpbiBkZXIgUHJvZHVrdGlvbnNsaW5pZSBcdTAwZTRuZGVydCBtXHUwMGZjc3N0ZSB3aWVkZXIgbmV1IHRyYWluaWVydCB3ZXJkZW4uIERhcyB3XHUwMGU0cmUgdGV1ZXIgdW5kIGF1ZndcdTAwZTRuZGlnLiJ9LCB7ImFuc3dlciI6ICJEYXMgTExNIHNvbGx0ZSBcdTAwZmNiZXIgUkFHIFp1Z3JpZmYgYXVmIFdhcnR1bmdzZGF0ZW4gdW5kIEhhbmRiXHUwMGZjY2hlciBiZWtvbW1lbi4iLCAiY29ycmVjdCI6IHRydWV9XX0='
</div>

<div class="alert alert-block alert-Info">
<b>QUIZ:</b> Führe die nächste Codezelle aus um die Quizfrage anzuzeigen. </div>

In [14]:

from helpers.q_helpers import load_question
from jupyterquiz import display_quiz
q1 = load_question('q1')
display_quiz([q1])


<IPython.core.display.Javascript object>

# Retrieval Augmented Generation (RAG)

Retrieval Augmented Generation (kurz: RAG, Deutsch: abruf-erweiterte Generation) ist eine Technik zur Verwendung von Large Language Models (LLMs). Die Anwendungsbeispiele für RAG sind sehr vielfältig. Grundsätzlich geht es darum, einem LLM spezielles Wissen bereitzustellen. 
Wir schaffen einen Mechanismus der in einer Textdatenbank nachschlagen kann. Die Ergebnisse dieser Suche werden dann an unser LLM übergeben. Das macht aus den gefundenen Textschnipseln eine wohlformulierte Antwort. 

![Platzhalter](pictures\RAG_LLM_diagramm.png)


So muss das LLM mit dem wir arbeiten nicht ständig nachtrainiert werden. Trotzdem kann neue Information über eine Datenbank verlässlich und elegant in den Generationsprozess miteinbezogen werden. In der Datenbank kann z.B. veraltete Information genauso problemlos gelöscht werden, wie neue Information hinzugefügt werden kann.




## 1 Unser Szenario 



<div class="alert alert-block alert-info"> <b> Das Unternehmen Alstom produziert Straßenbahnen. In der Produktionslinie B soll ein "Intelligenter Wartungsassistent" implementiert werden.
Dieser Assistent soll Technikern bei der Fehlerdiagnose und Wartung von Fertigungsrobotern und Montageanlagen helfen. Die Techniker geben ihre Beobachtung, oder Fehldercodes in ein Chatfester ein und der Wartungsassistent gibt Ihnen eine Handlungsempfehlung.</b> Diesen Wartungsassistenten wirst du im Laufe dieses Notebooks implementieren. </div>

![Platzhalter](pictures/Produktionslinie.jpg)



<div class="alert alert-block alert-Info">
<b>QUIZ:</b> Führe die nächste Codezelle aus, um die Quizfrage anzuzeigen. </div>


In [15]:
q2 = load_question('q2')
display_quiz([q2])

<IPython.core.display.Javascript object>


<div class="alert alert-block alert-info">
<b>Info:</b> Dies ist ein einmaliger Schritt. D.h er muss nur ausgeführt werden, wenn das Notebook auf einem neuen Computer läuft. Du kannst also direkt zum Importieren von Funktionen übergehen. </div>

In [16]:
#!pip install llama-index
#!pip install llama-index-embeddings-huggingface
#!pip install llama-index-llms-ollama

## 2.2 Importieren von Funktionen

 
Um einzelne Funktionen der Bibliothek nutzen zu können, müssen wir die benötigten Elemente von <strong>llama-index </strong> importieren. Die 5 Elemente <i> Settings, VectorStoreIndex, SimpleDirectoryReader, HuggingFaceEmbedding und ollama </i> werden wir im Laufe des Notebooks für unseren RAG-Wartungsassistenten verwenden.

In [17]:
from llama_index.core import Settings, VectorStoreIndex, SimpleDirectoryReader
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.ollama import Ollama
#from llama_index.core.callbacks import CallbackManager, LlamaDebugHandler

## 3.1 Suche nach Relevanten Daten

Jetzt geht es darum, die Wissensbasis zu kuratieren, also relevante Texte zu finden und sie dem Modell zugänglich zu machen. Denn unser RAG- Modul besteht aus zwei Teilen: einer Vektordatenbank, aka Wissensbasis, und einer Suchfunktion. 


![Platzhalter](pictures/RAG_Modul.png)



<div class="alert alert-block alert-success">
<b>Auftrag:</b> Überlege für ca. 5 Minuten welche Dokumente für den RAG-Wartungsassistenten sinnvoll oder notwendig wären! Notiert eure Überlegungen in dem Text Dokument, welches schon geöffnet ist. </div>

Normalerweise würdest du dich nach diesen Überlegungen nun selbst darum bemühen, die richtigen Dokumente zu finden und am richtigen Ort zu spreichen. In diesem Szenario haben wir das schon erledigt. Anstatt, dessen kommt ein Quiz zu Problemen die beim Befüllen der Wissensbasis auftreten können.

<div class="alert alert-block alert-Info">
<b>QUIZ:</b> Führe die nächste Codezelle aus, um die Quizfrage anzuzeigen. </div>


In [18]:
q4 = load_question('q4')
q5 = load_question('q5')
display_quiz([q4, q5])

<IPython.core.display.Javascript object>



Der Ordner <i> Wissensbasis </i> muss außerdem am richtigen Ort sein, damit unser RAG-System darauf zugreifen kann. 

<div class="alert alert-block alert-success">
<b>Auftrag:</b> Navigiere im Explorer in den Ordner ./Wissensbasis und seh dir für <b> ca. 10 Minuten </b> die Dokumente an, die dort schon liegen. Überlege dir beim Lesen der Dokumente schon ein paar Fragen, welche du dem RAG-Wartungsassistenen später stellen möchtest. Notiere auch diese im Word Dokument! </div>

## 3.2 Einlesen der Daten
Mit der nächsten Codezeile erhält das System Zugriff auf die Dateien welche in dem Ordner <i> Wissensbasis </i> gespreichert sind. Alle Dateien in diesem Ordner werden mit dem Objekt **documents** gleichgesetzt. So kann es anderen Funktionen zur Verfügung gestellt werden.

<div class="alert alert-block alert-success">
<b>Auftrag:</b> Führe die nächste Codezelle aus. </div>

In [19]:

documents = SimpleDirectoryReader("Wissensbasis").load_data()


<div class="alert alert-block alert-Info">
<b>QUIZ:</b> Führe die nächste Codezelle aus, um die Quizfrage anzuzeigen. Erinner dich an das Video aus dem Vorkurs: </div>



## 3.3 Embedding

Eben haben wir sichergestellt, dass die Dokumente in unserer Wissensbasis durchsuchbar sind. Das heißt eine suchfunktion kann Wörter und Sätze aus den Dokumenten entnehmen. Trotzdem liegen die Texte darin noch nicht im richtigen Format für ein LLM vor.<strong> Denn Neuronale Netze brauchen numerischen Input! </strong> Wir müssen also noch einen Umwandlungsschritt vorhemen: Jedes einzele enthaltene Wort muss in einen Vektor umgewandelt werden. 

![Platzhalter](../RAG/pictures/Embedding.png)

Dafür werden bereits trainierte Embedding Modelle verwendet. Die meisten Embedding Modelle sind einsprachig. Eininge große Modelle sind mehrsprachig, können aber trotzdem meistens eine besonders gut. 


<div class="alert alert-block alert-success">
<b>Auftrag:</b> Führe die nächste Codezelle aus, um das ein <b> Embedding-Modell </b> von der Plattform Huggingface zu importieren. </div>

In [20]:
Settings.embed_model = HuggingFaceEmbedding(model_name="mixedbread-ai/deepset-mxbai-embed-de-large-v1")


## 3.4 Speichern in Vektordatenbank

Nachdem das Embeddingmodell geladen wurde, kann es auf den Text in unserem **documents** Objekt angewendet werden. 
Mit der Nächste Codezeile passiert aber noch etwas mehr:

<ul>
<li> Jedem Wort wird ein Vektor zugeordnet (wie in der Abbildung oben) </li>
<li> Der ganze Text wird in kleine Schnipsel (Chunks) aufgeteilt. Default: 1024 Zeichen pro Chunk </li>
<li> Die Chunks werden dann wieder in einem Vektorraum (VektorStoreIndex) gespeichert. Dadurch sind sie Für die Suchfunktion die wir gleich zusammensatzen leichter durchsuchbar. </li>
</ul>

Grob können wir uns also vorstellen, dass jedes Chunk einen Punkt im Vektorraum zugeordnet bekommt. Auf diese Weise können die Chunks später mit Anfragen an das Modell verglichen werden.

![Platzhalter](pictures/Embedding_chunks.png)

 Hier ein Beispiel wie die Suchfunktion ein Mapping zwischen einer Useranfrage und den Chunks der Wissensdatenbank herstellt:
 Wenn ein User eine Frage stellt werden die einzelnen Worte auch durch das Embedding Modell in Vektoren umgewandelt und der ganzen Frage als Chunk ein Ort im der *VectorDataBase* zueordnet. Der Stern repräsentiert hier die Useranfrage. Durch den K-Nächste-Nachbarn Algorithmus (KNN) wird bestimmt welche Chunks aus der Datenbank abgefragt werden. Im Bild durch den Kreis gezeigt, siehst du welche Chunks bei k = 3 ausgewählt werden.

![Platzhalter](pictures/Embedding_search_.png)


<div class="alert alert-block alert-success">
<b>Auftrag: </b>Führe die nächste Codezelle aus. Um die Vektordatenbank zu befüllen.</div>

In [21]:
index = VectorStoreIndex.from_documents(
    documents,
)

## 4 Importieren des LLMs

In diesem Schritt geht es um die Auswahl und das Laden des passenden LLMs. Auch hier gibt es eigentlich wieder eine Vielzahl an Möglichkeiten, denn es gibt mittlerweile eine große Auswahl an Modellen. Das richtige Modell zu finden ist allerdings garnicht so leicht. Deshalb ist hier schon eines für dich vorausgewählt. Um nicht von der Internetverbindung anhänging zu sein und aus Datenschutzgründen wollten wir zunächst mal ein lokales Modell haben. D.h. es läuft direkt auf dem Rechner an dem du gerade arbeitst.
Wichtige Auswahlkriterien sind: 
<ul>
    <li> Es muss deutsch sprechen. </li>
    <li> Es muss auf Frage-Antwort Szenarien trainiert sein. </li>
    <li> Es muss klein genug sein, um auf unseren Rechnern laufen zu können. </li>
    <li> Es muss Open Source verfügbar sein.</li>
</ul>

Nach langer Recherche und viel Ausprobieren ist hier die Entscheidung auf das Modell <i> phi3 </i> gefallen.




<div class="alert alert-block alert-success">
<b>Auftrag:</b> Führe die nächste codezelle aus um das LLM zu laden </div> 

In [22]:
Settings.llm = Ollama(model="phi3", request_timeout=360.0)

## 5 Zusammensetzen, Ausprobieren, Evaluieren

Die Nächste Codezeile setzt nun alle Elemente die du bisher vorbereitet hast zusammen. D.h.: Es ist Zeit zum Ausprobieren! 

<div class="alert alert-block alert-success">
<b>Auftrag:</b> Überlege dir nun eine Anfrage an den RAG-Wartungsassistenten, oder wähle eine der Fragen die du dir vorhin notiert hast. <b> Wenn du die nächste Codezelle ausfüllst sieh an den oberen Bildschirmrand. Dort erscheint ein Feld indem du die Frage eingeben kannst! </b> Bestätige mit Enter um die Frage abzuschicken. </div>

<div style="display:none" class="alert alert-block alert-success">
<b>Auftrag:</b> Führe die nächste Codezelle aus. </div>

In [24]:
import textwrap
from helpers.query_helpers import set_query
TASK = 'WRITE'

#debug_handler = LlamaDebugHandler()
#callback_manager = CallbackManager([debug_handler])
query_engine = index.as_query_engine(
    #callback_manager = callback_manager
)

response = query_engine.query(set_query(TASK))
formatted_response=textwrap.fill(response.response, width=80)
print(formatted_response)

In einem Notfall sollte man sofort Maßnahmen ergreifen, um die Sicherheit aller
und das Risiko von Schäden zu minimieren. Die ersten Prioritäten liegen darin,
eine Personensicherheitsstufe einzufahren und Kontakt mit relevanten Personen
wie der Werkleitung oder dem Schichtleiter aufzunehmen. Es ist wichtig,
notwendige Sicherungsarbeiten an Maschinenelementen durchzuführen, wenn möglich.
Dabei sollte man sicherstellen, dass Fluchtwege frei bleiben und alle
erforderlichen Dokumentationen nach vollständiger Überprüfung der Situation
erstellt werden. Für spezielle Produktionsbereiche wie die Lackiererei oder
Roboterschweißstraße gibt es zusätzliche Maßnahmen, welche in einem Notfall
befolgt werden müssen, um weitergehende Schäden zu vermeiden und den
Arbeitsablauf nach der Stromwiederherstellung sicherzustellen. Bei Arbeiten an
Hochspannungskomponenten sollte zudem die sorgfältige Handhabung und
ausreichende Prüfungen durchgeführt werden, um Unfälle in diesem Bereich zu
vermeiden. In Notfa


<div class="alert alert-block alert-success"> Lese dir die Antwort des RAG-Wartungsassitenten <b> genau </b> durch, kopiere sie in das Word Dokument zu deiner Frage und beurteile die Qualität der Antwort! Notiere dir auch, anhand welcher Kriterien du die Qualität der Antworten beurteilt hast. 
</div>

## 6 Gegenprobe

Das Modell **Phi3** kannst du auch ohne RAG-Modul im Terminal verwenden. Wenn das Modell noch nicht aktiviert ist sieht der Terminal so aus:

![Platzhalter](pictures/Terminal_rag.png)

Um es zu starten musst du noch den Befehl *ollama run phi3* eintippen und mit *Enter* bestätigen.
Danach sieht der Terminal so aus und kannst direkt chatten.

![Platzhalter](pictures/Terminal_phi3_update.png)


<div class="alert alert-block alert-success">
<b>Aufgabe:</b> Stelle die selbe Frage die du eben an den RAG-Assistenten gestellt hast direkt an Phi3. Vergleiche die Antworten anhand folgender Kriterien:
</div>

<ul>
<li>Wird die Frage richtig beantwortet?</li>
<li>Wie präzise sind die antworten?</li>
</ul>


<div class="alert alert-block alert-success">
 <b> Wiederhole diesen Vorgang mit unterschieldichen Fragen </b>
</div>



<div style="display:none" >
    print("\nRetrieved Chunks:")
    for i, node in enumerate(debug_handler.get_retrived_nodes()):
    print(f"\n--- Chunk {i+1} ---")
    print(f"Text: {node.text}")

</div>